## This notebook is from llama_index, just trying it

In [1]:
!pip install llama-index llama-index-llms-huggingface llama-index-embeddings-huggingface transformers accelerate bitsandbytes llama-index-readers-web matplotlib flash-attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 9.2 MB/s eta 0:00:00:00:010:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 43.6 MB/s eta 0:00:0000:0100:01m
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 11.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 34.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### DATA

In [2]:
from llama_index.readers.web import BeautifulSoupWebReader

url = "https://www.theverge.com/2023/9/29/23895675/ai-bot-social-network-openai-meta-chatbots"
docs = BeautifulSoupWebReader().load_data([url])

### MODEL

In [3]:
from llama_index.llms.huggingface import HuggingFaceLLM


def messages_to_prompt(messages):
    prompt = ""
    system_found = False
    for message in messages:
        if message.role == "system":
            prompt += f"<|system|>\n{message.content}<|end|>\n"
            system_found = True
        elif message.role == "user":
            prompt += f"<|user|>\n{message.content}<|end|>\n"
        elif message.role == "assistant":
            prompt += f"<|assistant|>\n{message.content}<|end|>\n"
        else:
            prompt += f"<|user|>\n{message.content}<|end|>\n"

    # trailing prompt
    prompt += "<|assistant|>\n"

    if not system_found:
        prompt = (
            "<|system|>\nYou are a helpful AI assistant.<|end|>\n" + prompt
        )

    return prompt


llm = HuggingFaceLLM(
    model_name="microsoft/Phi-3-mini-4k-instruct",
    model_kwargs={
        "trust_remote_code": True,
    },
    generate_kwargs={"do_sample": True, "temperature": 0.1},
    tokenizer_name="microsoft/Phi-3-mini-4k-instruct",
    query_wrapper_prompt=(
        "<|system|>\n"
        "You are a helpful AI assistant.<|end|>\n"
        "<|user|>\n"
        "{query_str}<|end|>\n"
        "<|assistant|>\n"
    ),
    messages_to_prompt=messages_to_prompt,
    is_chat_model=True,
)

config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### EMBEDDINGS

In [4]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.llm = llm
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### INDEX

In [14]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(
    docs
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
from llama_index.core import SummaryIndex

summary_index = SummaryIndex.from_documents(
    docs
)

In [7]:
from llama_index.core.response.notebook_utils import display_response

In [17]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### BASIC RAG
1. compat
2. refine
3. tree_summarize

In [8]:
query_engine = vector_index.as_query_engine(
    response_mode = "compact"
)

response = query_engine.query(
    "How do OpenAI and Meta differ on AI tools?"
)

display_response(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-04-28 15:49:13.877743: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 15:49:13.877854: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 15:49:14.045436: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


**`Final Response:`** OpenAI and Meta differ in their approach to AI tools. OpenAI tends to present its products as productivity tools, focusing on simple utilities for getting things done. On the other hand, Meta, which is in the entertainment business, has developed its own uses for generative AI and voices, creating personality-driven chatbots for its messaging apps. While OpenAI's ChatGPT is a versatile AI assistant, Meta's AI characters are designed to interact with users in a more personalized and entertaining manner.

In [9]:
query_engine = vector_index.as_query_engine(
    response_mode = "refine"
)

response = query_engine.query(
    "How do OpenAI and Meta differ on AI tools?"
)

display_response(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

**`Final Response:`** OpenAI and Meta have distinct approaches to their AI tools. OpenAI primarily emphasizes the productivity aspect, showcasing their AI as straightforward utilities designed to help users complete tasks efficiently. Conversely, Meta, although also involved in AI development, leans towards the entertainment sector. They have introduced personality-driven chatbots for their messaging platforms, with a diverse range of celebrities lending their voices to these bots. This divergence in focus highlights the versatility and potential applications of AI technology across different industries.

In [10]:
query_engine = vector_index.as_query_engine(
    response_mode = "tree_summarize"
)

response = query_engine.query(
    "How do OpenAI and Meta differ on AI tools?"
)

display_response(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

**`Final Response:`** OpenAI and Meta differ in their approach and application of AI tools. OpenAI tends to present its products as productivity tools, focusing on simple utilities designed to help users accomplish tasks efficiently. Their AI tool, ChatGPT, is an example of this, providing users with a large language model that can generate human-like text responses, assist with various tasks, and even interact via voice.

On the other hand, Meta (formerly Facebook) is in the entertainment business and has developed its own uses for generative AI and voices. They have created 28 personality-driven chatbots for their messaging apps, with celebrities lending their voices to these AI characters. These chatbots are designed to engage users in a more personalized and entertaining manner, rather than focusing solely on productivity.

In summary, OpenAI emphasizes productivity and utility in their AI tools, while Meta focuses on entertainment and personalized interactions through their AI-driven chatbots.

### ROUTER QUERY ENGINE
1. single selector
2. multi selector

In [15]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    vector_index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts.",
    ),
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document.",
    ),
)

In [16]:
from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool], select_multi=False
)

response = query_engine.query("What was mentioned about Meta?")

display_response(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

**`Final Response:`** 1. Meta is in the entertainment business, not just the tech industry.
2. Meta has found its own uses for generative AI and voices.
3. Meta unveiled 28 personality-driven chatbots to be used in Meta's messaging apps.
4. Celebrities including Charli D'Amelio, Dwyane Wade, Kendall Jenner, MrBeast, Snoop Dogg, Tom Brady, and Paris Hilton lent their voices to Meta's chatbots.
5. Meta's chatbots have brief and often humorous character descriptions, such as MrBeast's characterization as "the big brother who will roast you."

In [18]:
from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    select_multi=True,
)

response = query_engine.query(
    "What was mentioned about Meta? Summarize with any other companies mentioned in the entire document."
)

display_response(response)

Selecting query engine 1: Useful for summarizing an entire document, which is needed to provide a summary about Meta and other companies mentioned in the document..


**`Final Response:`** Meta, a company in the entertainment business, is developing its own uses for generative AI and voices, as revealed on Wednesday. They unveiled 28 personality-driven chatbots to be used in Meta's messaging apps, with celebrity voices including Charli D'Amelio, Dwyane Wade, Kendall Jenner, MrBeast, Snoop Dogg, Tom Brady, and Paris Hilton. These chatbots come with brief and often cringeworthy descriptions, and are seen as an intermediate step towards a synthetic social network. Meta plans to place its AI characters on every major surface of its products, potentially transforming feeds from human-defined connections to partially synthetic social networks.

In addition to Meta, OpenAI, another tech company, is also developing AI tools. OpenAI presents its products as productivity tools, while Meta is focusing on entertainment. OpenAI's ChatGPT, a large language model, has evolved to become more useful and versatile, with the addition of a voice feature that could potentially lead to a more empathetic and patient AI companion. Both companies

### SUBQUESTION QUERY ENGINE

In [19]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    vector_index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts.",
    ),
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document.",
    ),
)

In [20]:
import nest_asyncio

nest_asyncio.apply()

In [21]:
from llama_index.core.query_engine import SubQuestionQueryEngine

query_engine = SubQuestionQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    verbose=True,
)

response = query_engine.query(
    "What was mentioned about Meta? How Does it differ from how OpenAI is talked about?"
)

display_response(response)

Generated 3 sub questions.
[vector_search] Q: What are the key points mentioned about Meta in documents?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[vector_search] A: 1. Meta is in the entertainment business, not just a productivity tool like OpenAI.
2. Meta has built its own uses for generative AI and voices.
3. Meta unveiled 28 personality-driven chatbots for its messaging apps.
4. Celebrities like Charli D'Amelio, Dwyane Wade, Kendall Jenner, MrBeast, Snoop Dogg, Tom Brady, and Paris Hilton lent their voices to these chatbots.
5. Each celebrity character has a brief and often humorous description, such as MrBeast's characterization as "the big brother who will roast you."
6. It's suggested that celebrities may not yet be willing to entrust their entire personas to Meta, but offering people a taste of interacting with AI Snoop Dogg could help iron out any issues before the full launch.
7. The introduction of these chatbots could potentially mark the beginning of a new era in social networking, where AI characters become a significant part of the user experience.
[vector_search] Q: What are the key points mentioned about OpenAI i

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[vector_search] A: 1. OpenAI announced the latest updates for ChatGPT, which now includes voice interaction and image uploading capabilities.
2. The addition of a voice feature to ChatGPT gives it a hint of personality, making it more powerful as a mobile app and potentially more empathetic and helpful.
3. OpenAI presents its products as productivity tools, but the development of generative AI and voices shows its interest in the entertainment industry.
4. OpenAI has created 28 personality-driven chatbots for use in Meta's messaging apps, with celebrity voices lending their personalities to these AI characters.
5. The voice feature is currently available to ChatGPT Plus subscribers, with free users expected to gain access in the future.
6. The potential for AI to become a synthetic social network is discussed, with the implications of a more personalized, engaging, and entertaining social media experience.
7. The development of AI in social networking raises questions about the range o

**`Final Response:`** 1. Meta is involved in the entertainment industry, creating personality-driven chatbots for messaging apps.
2. Celebrities have lent their voices to Meta's chatbots, offering users a unique and engaging experience.
3. Meta's chatbots could potentially lead to a new era in social networking, with AI characters becoming a significant part of the user experience.
4. The development of Meta's chatbots is seen as a step towards a partially synthetic social network.

In contrast, OpenAI is primarily focused on productivity tools, with its ChatGPT AI offering utilities like pep talks and voice memos. While OpenAI also develops generative AI and voices, its applications are more centered around utility rather than entertainment. The key difference lies in Meta's emphasis on creating an engaging and entertaining social networking experience through celebrity-driven AI chatbots, while OpenAI's ChatGPT is geared towards productivity and utility.